In [206]:
import json
import pandas as pd
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import wikipedia

In [207]:
vids = pd.read_csv('labeled_vids.csv')

In [222]:
vids.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)

,date,title,class,viewCount,commentCount,likeCount,dislikeCount,ratio,sentiment,magnitude
0,2013-01-17,Joe Rogan Experience #1 - Brian Redban,2,673255,2091,6485,175,37.057143,0.175496,32.058786
1,2013-01-17,Joe Rogan Experience #2 - Brian Redban,2,134693,359,1004,68,14.764706,0.136606,36.037951
2,2013-01-17,Joe Rogan Experience #3 - Ari Shaffir,2,133176,320,1016,66,15.393939,0.145467,31.501767
3,2013-01-17,Joe Rogan Experience #5 - Ari Shaffir & John H...,2,49565,59,343,19,18.052632,0.102881,46.788983
4,2013-01-18,Joe Rogan Experience #5 - Ari Shaffir & John H...,2,33535,48,278,11,25.272727,0.175208,35.346667
5,2013-01-18,Joe Rogan Experience #6 - Brian Redban,2,39780,104,399,19,21.000000,0.143370,38.702500
6,2013-01-18,Joe Rogan Experience #7 - Brian Redban,2,32255,58,316,13,24.307692,0.115000,35.839655
7,2013-01-18,Joe Rogan Experience #9 - Brian Redban (Part 1),2,20522,29,217,8,27.125000,0.121379,25.518621
8,2013-01-18,Joe Rogan Experience #9 - Brian Redban (Part 2),2,14343,28,197,6,32.833333,0.044286,27.581786
9,2013-01-18,Joe Rogan Experience #11 - Ari Shaffir,2,25240,39,251,7,35.857143,0.109231,34.534615
